[View in Colaboratory](https://colab.research.google.com/github/opooladz/ESPRIT-Autoencoder-Anomaly-Detection/blob/master/Copy_of_ESPRIT2.ipynb)

In [0]:
!apt-get install nvidia-cuda-toolkit
!pip3 install numba

import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/lib/nvidia-cuda-toolkit/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/lib/x86_64-linux-gnu/libnvvm.so"

from numba import cuda
import numpy as np
import time

@cuda.jit
def hello(data):
    data[cuda.blockIdx.x, cuda.threadIdx.x] = cuda.blockIdx.x

numBlocks = 5
threadsPerBlock = 10

data = np.ones((numBlocks, threadsPerBlock), dtype=np.uint8)

hello[numBlocks, threadsPerBlock](data)

print(data)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  accountsservice acpid activity-log-manager adwaita-icon-theme apg aspell
  aspell-en at-spi2-core avahi-daemon avahi-utils bbswitch-dkms bind9-host
  binfmt-support bluez bluez-obexd bsdmainutils ca-certificates-java
  cheese-common clang-3.8 cracklib-runtime crda cups-pk-helper dbus
  dbus-user-session dbus-x11 dconf-cli dconf-gsettings-backend dconf-service
  default-jre default-jre-headless desktop-file-utils dictionaries-common
  distro-info-data dkms dmsetup dns-root-data dnsmasq-base dosfstools eject
  emacsen-common enchant evolution-data-server evolution-data-server-common
  fakeroot file fontconfig fonts-dejavu-extra fuse gcr gdisk geoclue
  geoclue-ubuntu-geoip geoip-database gettext-base gir1.2-atk-1.0
  gir1.2-freedesktop gir1.2-gdkpixbuf-2.0 gir1.2-glib-2.0 gir1.2-gtk-3.0
  gir1.2-ibus-1.0 gir1.2-notify-0.7 gir1.2-packagekit

In [0]:
!pip install numba

    100% |████████████████████████████████| 1.9MB 7.9MB/s 
    100% |████████████████████████████████| 15.9MB 2.5MB/s 


In [1]:
import numpy as np
from scipy.linalg import toeplitz
from numpy import linalg as lg
from time import time
from typing import Tuple
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from numba import jit

flatten = lambda l: [item for sublist in l for item in sublist]

@jit
def compute_autocovariance(x: np.ndarray, M: int) -> np.ndarray:
    """
    This function compute the auto-covariance matrix of a numpy signal.
    The auto-covariance is computed as follows
    .. math:: \textbf{R}=\frac{1}{N}\sum_{M-1}^{N-1}\textbf{x}_{m}\textbf{x}_{m}^{H}
    where :math:`\textbf{x}_{m}^{T}=[x[m],x[m-1],x[m-M+1]]`.
    :param x: 1-D vector of size N
    :param M:  int, optional. Size of signal block.
    :returns: NxN ndarray
    """

    # Create covariance matrix for psd estimation
    # length of the vector x
    x = np.asarray(x).squeeze()
    assert x.ndim == 1, '1-D only'
    N = x.size

    # Create column vector (Nx1) from row array
    x_vect = x[None, :].T

    # init covariance matrix
    yn = x_vect[M-1::-1]  # reverse order from M-1 to 0

    R = yn @ yn.conj().T  # zeroth lag
    # about 5-8% of computation time
    for i in range(1, N-M):  # no zero because we just computed it
        # extract the column vector
        yn = x_vect[M-1+i:i-1:-1]

        R = R + yn @ yn.conj().T

    return R / N
@jit
def wrapper(x):
  return np.convolve(x,xc,mode="valid")

@jit
def esprit1(x: np.ndarray, L: int, M: int=None, fs: int=1,
           verbose: bool=False) -> Tuple[np.ndarray, np.ndarray]:
    """
    This function estimate the frequency components based on the ESPRIT algorithm [ROY89]_
    The frequencies are related to the roots as :math:`z=e^{-2j\pi f/Fe}`.
    See [STO97]_ section 4.7 for more information about the implementation.
    :param x: ndarray, Nsamples
    :param L: int. Number of components to be extracted.
    :param M:  int, optional. Size of signal block.
    :param Fs: float. Sampling Frequency.
    :returns: ndarray ndarray containing the L frequencies
    >>> import numpy as np
    >>> import spectral_analysis.spectral_analysis as sa
    >>> Fe=500
    >>> t=1.*np.arange(100)/Fe
    >>> x=np.exp(2j*np.pi*55.2*t)
    >>> f=sa.Esprit(x,1,None,Fe)
    >>> print(f)
    """

    x = np.asarray(x).squeeze()
    assert x.ndim in (1, 2)
    # length of the vector x
    if x.ndim == 1:
        N = x.size
    else:
        N = x.shape[1]

    if M is None:
        M = N // 2
# %% extract signal subspace  99.9 % of computation time
    tic = time()
    if x.ndim == 1 and isinstance(M, int):
        R = compute_autocovariance(x, M)  # 75% of computation time        
    else:
        # the random phase of transmit/receive/target actually helps--need at least 5-6 observations to make useful
        R = np.cov(x, rowvar=False)
    if verbose:
        print('autocov sec.', time()-tic)
    # R = subspace.corrmtx(x.astype(complex128),M).astype(float) #f2py fortran

    tic = time()
    #U, S, V = lg.svd(R)  # 25% of computation time
    w, v = lg.eig(R)    
    idx = w.argsort()[::-1]   
    w = w[idx]
    v = v[:,idx]    
    if verbose:
        print('svd sec.', time()-tic)
# %% take eigenvalues and determine sinusoid frequencies
    # Remove last row
    S1 = v[:-1, :L]
    # Remove first row
    S2 = v[1:, :L]

    # Compute matrix Phi (Stoica 4.7.12)  <0.1 % of computation time
    Phi = lg.inv(S1.conj().T @ S1) @ S1.conj().T @ S2

    # Perform eigenvalue decomposition <0.1 % of computation time
    V, U = lg.eig(Phi)

    # extract frequencies ((note that there a minus sign since Yn are defined as [y(n), y(n-1),y(n-2),..].T))
    ang = -np.angle(V)

    # frequency normalisation
    f = fs*ang / (2.*np.pi)
    t = np.arange(0, 0.01, 1/fs)
    
    x2 = np.exp(-1j*2*np.pi*t[::-1])[:,np.newaxis]**f
    #display(xc.shape)
    #ampCisoid = np.apply_along_axis(lambda q: np.convolve(q,xc,mode="valid"),axis = 0,arr=x2)
    ampCisoid = np.apply_along_axis(wrapper,axis = 0,arr=x2)
    return f, np.abs(w[:L]), np.abs(ampCisoid)*1./480., np.angle(ampCisoid)
  

def esprit2(x: np.ndarray, L: int, M: int=None, fs: int=1,
           verbose: bool=False) -> Tuple[np.ndarray, np.ndarray]:
    """
    This function estimate the frequency components based on the ESPRIT algorithm [ROY89]_
    The frequencies are related to the roots as :math:`z=e^{-2j\pi f/Fe}`.
    See [STO97]_ section 4.7 for more information about the implementation.
    :param x: ndarray, Nsamples
    :param L: int. Number of components to be extracted.
    :param M:  int, optional. Size of signal block.
    :param Fs: float. Sampling Frequency.
    :returns: ndarray ndarray containing the L frequencies
    >>> import numpy as np
    >>> import spectral_analysis.spectral_analysis as sa
    >>> Fe=500
    >>> t=1.*np.arange(100)/Fe
    >>> x=np.exp(2j*np.pi*55.2*t)
    >>> f=sa.Esprit(x,1,None,Fe)
    >>> print(f)
    """

    x = np.asarray(x).squeeze()
    assert x.ndim in (1, 2)
    # length of the vector x
    if x.ndim == 1:
        N = x.size
    else:
        N = x.shape[1]

    if M is None:
        M = N // 2
# %% extract signal subspace  99.9 % of computation time
    tic = time()
    if x.ndim == 1 and isinstance(M, int):
        R = compute_autocovariance(x, M)  # 75% of computation time
    else:
        # the random phase of transmit/receive/target actually helps--need at least 5-6 observations to make useful
        R = np.cov(x, rowvar=False)
    if verbose:
        print('autocov sec.', time()-tic)
    # R = subspace.corrmtx(x.astype(complex128),M).astype(float) #f2py fortran

    tic = time()
    U, S, V = lg.svd(R)  # 25% of computation time
    #w, v = lg.eig(R)    
    #idx = w.argsort()[::-1]   
    #w = w[idx]
    #v = v[:,idx]    
    if verbose:
        print('svd sec.', time()-tic)
# %% take eigenvalues and determine sinusoid frequencies
    # Remove last row
    S1 = U[:-1, :L]
    # Remove first row
    S2 = U[1:, :L]

    # Compute matrix Phi (Stoica 4.7.12)  <0.1 % of computation time
    Phi = lg.inv(S1.conj().T @ S1) @ S1.conj().T @ S2

    # Perform eigenvalue decomposition <0.1 % of computation time
    V, U = lg.eig(Phi)

    # extract frequencies ((note that there a minus sign since Yn are defined as [y(n), y(n-1),y(n-2),..].T))
    ang = -np.angle(V)

    # frequency normalisation
    f = fs*ang / (2.*np.pi)

    return f, S[:L]

Using TensorFlow backend.


In [0]:
from time import time
import numpy as np
from pandas import DataFrame

# Bool: Needs to be set by user 
full_anom = 1
anom_samples = 0

x = np.random.randn(4096).astype(np.complex128)
F = 12345.6

f0 = 12345.6
f1 = 1000.6
f2 = 726.6
f3 = 57.3

f0=np.random.uniform(low=0.,high = 10000.)
f1=np.random.uniform(low=0.,high = 10000.)
f2=np.random.uniform(low=0.,high = 10000.)
f3 = np.random.uniform(low=0.,high = 10000.)


c0 = 15.
c1 = 7.
c2 = 10.
c0= np.random.uniform(low=0.,high = 100.)
c1= np.random.uniform(low=0.,high = 100.)
c2= np.random.uniform(low=0.,high = 100.)

fs = 48e3
snr = 20.  # dB
Ntone = 6

t = np.arange(0, 0.01, 1/fs)

nvar = 10**(-snr/10.)



M = [100] * 200  # iterating over block length

py = DataFrame(index=M, columns=['err', 'sigma','cisoidAmp','cisoidAngle'])
fortreal = DataFrame(index=M, columns=['err', 'sigma','cisoidAmp','cisoidAngle'])
fortcmpl = DataFrame(index=M, columns=['err', 'sigma','cisoidAmp','cisoidAngle'])
fest = []
fest2 = []
cisoidAmp = []
cisoidAmp2 = []
i = 0
for m in M:
    i = i + 1
    # Generate fist tripplet of cisoid  
    # Train data start
    xc = c0*np.exp(1j*2*np.pi*f0*t)+c1*np.exp(1j*2*np.pi*f1*t) +c2*np.exp(1j*2*np.pi*f2*t)
    xc1 = xc + np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
  
    tmpFest, sigma,tmpAmp,cisoidAngle = esprit1(xc1, Ntone//2, M=m, fs=fs, verbose=False)
    fest.append(tmpFest)
    tmpAmp = tmpAmp[0]
    cisoidAmp.append(tmpAmp)
    # Train data end 
    
    # Test section start
    
    # Inject Anomaly
    if not(full_anom or i%20):
      display(i)
      for j in np.arange(anom_samples):
        display(i)
        xc2 = c0*np.exp(1j*2*np.pi*f0*t)+c1*np.exp(1j*2*np.pi*f1*t) +c2*np.exp(1j*2*np.pi*f3*t)+ np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
        tmpFest, sigma,tmpAmp,cisoidAngle = esprit1(xc2, Ntone//2, M=m, fs=fs, verbose=False)
        fest2.append(tmpFest)
        tmpAmp = tmpAmp[0]
        cisoidAmp2.append(tmpAmp) 
    # When not Anomaly use xc as baseline signal and add AWGN - Run through esprit
    xc2 = xc +  np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
    if full_anom:
      xc2 = c0*np.exp(1j*2*np.pi*f0*t)+c1*np.exp(1j*2*np.pi*f1*t) +c2*np.exp(1j*2*np.pi*f3*t)+ np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
    tmpFest, sigma,tmpAmp,cisoidAngle = esprit1(xc2, Ntone//2, M=m, fs=fs, verbose=False)
    fest2.append(tmpFest)
    tmpAmp = tmpAmp[0]
    cisoidAmp2.append(tmpAmp)  
    # Test section end 

# Create train and test set     
x_train = np.hstack((cisoidAmp,fest))    
x_test = np.hstack((cisoidAmp2,fest2))
    

In [53]:
display(cisoidAmp2)
display(cisoidAmp)



[array([81.31039792,  0.36678567, 36.92377744]),
 array([81.31039326,  0.3667165 , 36.92844429]),
 array([81.31038561,  0.36686799, 36.90910084]),
 array([81.31043276,  0.3667679 , 36.92275643]),
 array([81.31035601,  0.36670495, 36.95811756]),
 array([81.31043669,  0.36678596, 36.93075044]),
 array([81.31043314,  0.36681616, 36.92469977]),
 array([81.31043612,  0.36676893, 36.92213671]),
 array([81.31045292,  0.36674938, 36.90080309]),
 array([81.31043506,  0.36688442, 36.93206789]),
 array([81.31042434,  0.36693525, 36.918718  ]),
 array([81.3104352 ,  0.366804  , 36.93293647]),
 array([81.3104218 ,  0.36684977, 36.93148104]),
 array([81.3103845 ,  0.36677581, 36.92958243]),
 array([81.31043724,  0.36672686, 36.91589681]),
 array([81.31042329,  0.36680944, 36.90782101]),
 array([81.31037596,  0.366778  , 36.92740982]),
 array([81.31043825,  0.36677565, 36.93076363]),
 array([81.31038415,  0.36676112, 36.90775865]),
 array([81.31047836,  0.3667402 , 36.92543052]),
 array([81.31041121,

[array([81.31041386, 60.34952264, 36.87876563]),
 array([81.31040942, 60.3495108 , 37.02317559]),
 array([81.31038433, 60.3494674 , 36.90983775]),
 array([81.31041443, 60.34939621, 36.73013733]),
 array([81.31045641, 60.34965556, 36.98536513]),
 array([81.31041578, 60.34957406, 36.88451636]),
 array([81.31040104, 60.34954435, 36.89041638]),
 array([81.31038457, 60.34941213, 36.80694291]),
 array([81.31040584, 60.34962407, 37.03242845]),
 array([81.31040596, 60.34920747, 37.03735496]),
 array([81.31045648, 60.3492801 , 36.92096976]),
 array([81.31047848, 60.34922344, 36.87104944]),
 array([81.31040044, 60.34945419, 36.8465765 ]),
 array([81.31042765, 60.34938749, 36.82301745]),
 array([81.31041697, 60.34945493, 37.10090361]),
 array([81.31040492, 60.34894867, 37.11282097]),
 array([81.31039846, 60.34938857, 36.80649764]),
 array([81.31040412, 60.34935963, 37.0422359 ]),
 array([81.31038372, 60.34957003, 36.93368043]),
 array([81.31037824, 60.34958247, 36.77147092]),
 array([81.31037585,

In [52]:
display(fest2)
display(fest)



[array([1488.15724048, 6366.04193842, 8028.86058548]),
 array([1488.15778998, 6366.04705907, 8028.86722486]),
 array([1488.15868961, 6366.03584518, 8028.83970654]),
 array([1488.15310344, 6366.0432536 , 8028.85913294]),
 array([1488.16214763, 6366.04791367, 8028.90944361]),
 array([1488.15263311, 6366.04191668, 8028.87050581]),
 array([1488.15305829, 6366.03968152, 8028.86189763]),
 array([1488.15270229, 6366.04317783, 8028.85825131]),
 array([1488.15068497, 6366.04462451, 8028.82790282]),
 array([1488.1528281 , 6366.03462947, 8028.87238014]),
 array([1488.15410836, 6366.03086796, 8028.85338775]),
 array([1488.15281162, 6366.04058186, 8028.87361588]),
 array([1488.1544111 , 6366.03719396, 8028.87154522]),
 array([1488.15881964, 6366.04266807, 8028.86884407]),
 array([1488.15256747, 6366.04629188, 8028.84937431]),
 array([1488.1542335 , 6366.04017914, 8028.83788593]),
 array([1488.15982147, 6366.04250655, 8028.86575314]),
 array([1488.1524473 , 6366.04268049, 8028.87052456]),
 array([14

[array([1488.15535521, 8083.4990012 , 8028.79655639]),
 array([1488.1558818 , 8083.49424433, 8029.00202913]),
 array([1488.15883963, 8083.47809173, 8028.84075484]),
 array([1488.15528752, 8083.45480486, 8028.58523096]),
 array([1488.15026437, 8083.57630168, 8028.94821657]),
 array([1488.15512722, 8083.5220697 , 8028.804736  ]),
 array([1488.15687235, 8083.50820123, 8028.81312819]),
 array([1488.15881229, 8083.45973498, 8028.69441761]),
 array([1488.15630558, 8083.55076556, 8029.01519955]),
 array([1488.1562913 , 8083.40445974, 8029.02221212]),
 array([1488.15025489, 8083.4223771 , 8028.85659116]),
 array([1488.14759049, 8083.4082754 , 8028.78558154]),
 array([1488.15694355, 8083.47350898, 8028.75077605]),
 array([1488.15371453, 8083.45216415, 8028.71727411]),
 array([1488.15498567, 8083.47376122, 8029.11268549]),
 array([1488.15641348, 8083.34980418, 8029.12965538]),
 array([1488.15717769, 8083.45249002, 8028.69378451]),
 array([1488.15650869, 8083.44397607, 8029.02916001]),
 array([14

In [42]:
from keras import regularizers
from sklearn.preprocessing import StandardScaler
# Make sure to run this after each new generation of data
# zero mean and unit var
x_train2 = StandardScaler().fit_transform(x_train)
x_test2 = StandardScaler().fit_transform(x_test)
#x_train2 = x_train2[:,:,np.newaxis]
#x_test2 = x_test2[:,:,np.newaxis]

#x_train2 = x_train2.astype('float32') / 255.
#x_test2 = x_test2.astype('float32') / 255.

display(x_train2.shape)

input_dim = x_train.shape[1]

# set size of autoencoder 
encoding_dim = 6

# use elu because it is leaky tried both net and l1 and l2 : net and l1 worked the best 
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="elu",kernel_initializer= 'he_normal', activity_regularizer=regularizers.l1_l2(l1=10e-5, l2=10e-4))(input_layer)
#encoder = Dense(encoding_dim, activation="elu",activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="elu",kernel_initializer= 'he_normal')(encoder)
decoder = Dense(int(encoding_dim / 2), activation='elu',kernel_initializer= 'he_normal')(encoder)
decoder = Dense(input_dim, activation='elu',kernel_initializer= 'he_normal')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

autoencoder.summary()  


(200, 6)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dense_25 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_26 (Dense)             (None, 3)                 21        
_________________________________________________________________
dense_27 (Dense)             (None, 3)                 12        
_________________________________________________________________
dense_28 (Dense)             (None, 6)                 24        
Total params: 99
Trainable params: 99
Non-trainable params: 0
_________________________________________________________________


In [5]:
#Results with no anomily 
# Loss and Val Loss are very close + Accuracy and Val Accuracy are very close 
# Depending on the params of the net it can hit Accuracy of  1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
full_anom = 0
anom_samples = 0
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    #loss = 'binary_crossentropy',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy_with_masking', factor=0.2,
                              patience=5,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])
score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)
display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 0s 2ms/step - loss: 0.5229 - acc: 0.6167 - val_loss: 0.6079 - val_acc: 0.7500
Epoch 2/800
180/180 [==============================] - 0s 278us/step - loss: 0.5127 - acc: 0.6278 - val_loss: 0.5990 - val_acc: 0.7500
Epoch 3/800
180/180 [==============================] - 0s 307us/step - loss: 0.5022 - acc: 0.6333 - val_loss: 0.5901 - val_acc: 0.7500
Epoch 4/800
180/180 [==============================] - 0s 290us/step - loss: 0.4927 - acc: 0.6333 - val_loss: 0.5807 - val_acc: 0.7000
Epoch 5/800
 32/180 [====>.........................] - ETA: 0s - loss: 0.4667 - acc: 0.7500

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:972: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_accuracy_with_masking` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


180/180 [==============================] - 0s 320us/step - loss: 0.4829 - acc: 0.6389 - val_loss: 0.5705 - val_acc: 0.7000
Epoch 6/800
180/180 [==============================] - 0s 299us/step - loss: 0.4734 - acc: 0.6333 - val_loss: 0.5620 - val_acc: 0.6500
Epoch 7/800
180/180 [==============================] - 0s 326us/step - loss: 0.4632 - acc: 0.6556 - val_loss: 0.5519 - val_acc: 0.6500
Epoch 8/800
180/180 [==============================] - 0s 373us/step - loss: 0.4528 - acc: 0.6611 - val_loss: 0.5431 - val_acc: 0.6500
Epoch 9/800
180/180 [==============================] - 0s 320us/step - loss: 0.4432 - acc: 0.6667 - val_loss: 0.5324 - val_acc: 0.6500
Epoch 10/800
180/180 [==============================] - 0s 312us/step - loss: 0.4334 - acc: 0.6833 - val_loss: 0.5234 - val_acc: 0.6500
Epoch 11/800
180/180 [==============================] - 0s 306us/step - loss: 0.4246 - acc: 0.7000 - val_loss: 0.5132 - val_acc: 0.7000
Epoch 12/800
180/180 [==============================] - 0s 313us/

'[test_loss, test_acc]'

[0.11783276438713074, 0.945]

In [54]:
from keras import regularizers
from sklearn.preprocessing import StandardScaler
# Make sure to run this after each new generation of data
# zero mean and unit var
x_train2 = StandardScaler().fit_transform(x_train)
x_test2 = StandardScaler().fit_transform(x_test)
#x_train2 = x_train2[:,:,np.newaxis]
#x_test2 = x_test2[:,:,np.newaxis]

#x_train2 = x_train2.astype('float32') / 255.
#x_test2 = x_test2.astype('float32') / 255.

display(x_train2.shape)

input_dim = x_train.shape[1]

# set size of autoencoder 
encoding_dim = 6
elu = keras.layers.ELU(alpha=1.2)
lrlu= keras.layers.LeakyReLU(alpha=0.3)


# use elu because it is leaky tried both net and l1 and l2 : net and l1 worked the best 
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim,kernel_initializer= 'he_normal', activity_regularizer=regularizers.l1_l2(l1=10e-5, l2=75.10e-5))(input_layer)
encoder = keras.layers.LeakyReLU(alpha=0.011)(encoder)
#encoder = Dense(encoding_dim, activation="elu",activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2),kernel_initializer= 'he_normal')(encoder)
encoder = keras.layers.LeakyReLU(alpha=0.011)(encoder)
decoder = Dense(int(encoding_dim / 2),kernel_initializer= 'he_normal')(encoder)
decoder = keras.layers.LeakyReLU(alpha=0.011)(decoder)
decoder = Dense(input_dim,kernel_initializer= 'he_normal')(decoder)
decoder = keras.layers.LeakyReLU(alpha=0.011)(decoder)
autoencoder2 = Model(inputs=input_layer, outputs=decoder)

autoencoder2.summary()  


(200, 6)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dense_33 (Dense)             (None, 6)                 42        
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 6)                 0         
_________________________________________________________________
dense_34 (Dense)             (None, 3)                 21        
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 3)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 3)                 12        
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 3)                 0         
__________

In [0]:
#Results with no anomily 
# Loss and Val Loss are very close + Accuracy and Val Accuracy are very close 
# Depending on the params of the net it can hit Accuracy of  1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
full_anom = 0
anom_samples = 0
batch_size = 32
import keras

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
nadam =keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

autoencoder2.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    #loss = 'binary_crossentropy',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder2.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])
score = autoencoder2.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)
display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 7s 38ms/step - loss: 0.6340 - acc: 0.5222 - val_loss: 0.6486 - val_acc: 0.4000
Epoch 2/800
180/180 [==============================] - 0s 363us/step - loss: 0.6323 - acc: 0.5167 - val_loss: 0.6465 - val_acc: 0.4000
Epoch 3/800
180/180 [==============================] - 0s 358us/step - loss: 0.6305 - acc: 0.5333 - val_loss: 0.6434 - val_acc: 0.4000
Epoch 4/800
180/180 [==============================] - 0s 359us/step - loss: 0.6292 - acc: 0.5389 - val_loss: 0.6430 - val_acc: 0.4000
Epoch 5/800
180/180 [==============================] - 0s 364us/step - loss: 0.6275 - acc: 0.5611 - val_loss: 0.6400 - val_acc: 0.4000
Epoch 6/800
180/180 [==============================] - 0s 347us/step - loss: 0.6262 - acc: 0.5778 - val_loss: 0.6392 - val_acc: 0.4500
Epoch 7/800
180/180 [==============================] - 0s 354us/step - loss: 0.6250 - acc: 0.6111 - val_loss: 0.6368 - val_acc: 0.5000
Epoch 8/800

'[test_loss, test_acc]'

[0.43619103610515597, 0.945]

In [56]:

#Results with full anomily 
full_anom = 1 
anom_samples = 0
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=200,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 1s 6ms/step - loss: 0.1577 - acc: 0.8167 - val_loss: 0.1850 - val_acc: 0.8000
Epoch 2/800
180/180 [==============================] - 0s 269us/step - loss: 0.1542 - acc: 0.8278 - val_loss: 0.1834 - val_acc: 0.8500
Epoch 3/800
180/180 [==============================] - 0s 255us/step - loss: 0.1527 - acc: 0.8111 - val_loss: 0.1818 - val_acc: 0.9000
Epoch 4/800
180/180 [==============================] - 0s 253us/step - loss: 0.1573 - acc: 0.8333 - val_loss: 0.1803 - val_acc: 0.9000
Epoch 5/800
180/180 [==============================] - 0s 246us/step - loss: 0.1521 - acc: 0.8444 - val_loss: 0.1807 - val_acc: 0.9000
Epoch 6/800
180/180 [==============================] - 0s 309us/step - loss: 0.1517 - acc: 0.8333 - val_loss: 0.1817 - val_acc: 0.9000
Epoch 7/800
180/180 [==============================] - 0s 265us/step - loss: 0.1519 - acc: 0.8556 - val_loss: 0.1794 - val_acc: 0.8500
Epoch 8/800


'[test_loss, test_acc]'

[0.6634244680404663, 0.565]

In [46]:

#Results with full anomily 
full_anom = 1 
anom_samples = 0
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 1s 4ms/step - loss: 0.1152 - acc: 0.8667 - val_loss: 0.1742 - val_acc: 0.7000
Epoch 2/800
180/180 [==============================] - 0s 280us/step - loss: 0.1145 - acc: 0.8611 - val_loss: 0.1704 - val_acc: 0.7500
Epoch 3/800
180/180 [==============================] - 0s 265us/step - loss: 0.1134 - acc: 0.8667 - val_loss: 0.1692 - val_acc: 0.8000
Epoch 4/800
180/180 [==============================] - 0s 276us/step - loss: 0.1138 - acc: 0.8611 - val_loss: 0.1701 - val_acc: 0.7000
Epoch 5/800
180/180 [==============================] - 0s 281us/step - loss: 0.1147 - acc: 0.8556 - val_loss: 0.1743 - val_acc: 0.7500
Epoch 6/800
180/180 [==============================] - 0s 267us/step - loss: 0.1144 - acc: 0.8556 - val_loss: 0.1706 - val_acc: 0.7500
Epoch 7/800
180/180 [==============================] - 0s 281us/step - loss: 0.1145 - acc: 0.8556 - val_loss: 0.1695 - val_acc: 0.8000
Epoch 8/800


'[test_loss, test_acc]'

[28.785220642089843, 0.215]

In [48]:

#Results with full anomily 
full_anom = 1 
anom_samples = 0
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.predict( x_test2, batch_size=32, verbose=1, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 1s 5ms/step - loss: 0.1100 - acc: 0.8722 - val_loss: 0.1565 - val_acc: 0.8000
Epoch 2/800
180/180 [==============================] - 0s 271us/step - loss: 0.1080 - acc: 0.8778 - val_loss: 0.1583 - val_acc: 0.7500
Epoch 3/800
180/180 [==============================] - 0s 251us/step - loss: 0.1083 - acc: 0.8556 - val_loss: 0.1551 - val_acc: 0.8500
Epoch 4/800
180/180 [==============================] - 0s 255us/step - loss: 0.1080 - acc: 0.8500 - val_loss: 0.1556 - val_acc: 0.7500
Epoch 5/800
180/180 [==============================] - 0s 247us/step - loss: 0.1096 - acc: 0.8556 - val_loss: 0.1568 - val_acc: 0.8500
Epoch 6/800
180/180 [==============================] - 0s 273us/step - loss: 0.1080 - acc: 0.8778 - val_loss: 0.1582 - val_acc: 0.8500
Epoch 7/800
180/180 [==============================] - 0s 258us/step - loss: 0.1086 - acc: 0.8611 - val_loss: 0.1555 - val_acc: 0.8500
Epoch 8/800


'[test_loss, test_acc]'

array([[-0.8869843 , -0.990225  ,  2.1923428 , -0.9147169 ,  4.2005634 ,
         2.1977317 ],
       [-0.99992776, -1.        ,  5.055438  , -0.99997467, 23.499792  ,
         5.073369  ],
       [-0.88610065, -0.9999998 ,  4.136735  , -0.91810477, 14.903526  ,
         4.150808  ],
       ...,
       [ 0.43674803,  3.4226084 , -0.1418013 ,  0.43208003, -0.97543687,
        -0.14499533],
       [ 0.5185466 ,  3.286262  , -0.7900079 ,  0.51922584, -0.9707997 ,
        -0.79176426],
       [-0.9999703 , -1.        ,  6.031487  , -0.9999908 , 32.733353  ,
         6.054444  ]], dtype=float32)

In [49]:
display(score[0,:])
display(x_test2[0,:])
display(x_train2[0,:])
display(x_train[0,:])

array([-0.8869843, -0.990225 ,  2.1923428, -0.9147169,  4.2005634,
        2.1977317], dtype=float32)

array([-0.76201746,  0.1862485 ,  1.02741918, -0.78946793,  0.18541389,
        1.02776708])

array([-1.07551739,  0.85003167, -0.41483063, -1.07819301, -0.84927881,
       -0.41709346])

array([1.50334274e+01, 9.63736662e+00, 6.49451948e+00, 1.23455774e+04,
       7.26571006e+02, 1.00057878e+03])

In [0]:
#Results with one sample anomily 
# the duration of the anom can be tuned with anom_samples
full_anom = 0
anom_samples = 1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 9s 48ms/step - loss: 1.5007 - acc: 0.1944 - val_loss: 1.4750 - val_acc: 0.0500
Epoch 2/800
180/180 [==============================] - 0s 353us/step - loss: 1.4188 - acc: 0.1778 - val_loss: 1.3748 - val_acc: 0.1000
Epoch 3/800
180/180 [==============================] - 0s 345us/step - loss: 1.3436 - acc: 0.1778 - val_loss: 1.2763 - val_acc: 0.1000
Epoch 4/800
180/180 [==============================] - 0s 350us/step - loss: 1.2729 - acc: 0.1889 - val_loss: 1.1892 - val_acc: 0.1000
Epoch 5/800
180/180 [==============================] - 0s 372us/step - loss: 1.2061 - acc: 0.2000 - val_loss: 1.1073 - val_acc: 0.0500
Epoch 6/800
180/180 [==============================] - 0s 348us/step - loss: 1.1525 - acc: 0.2222 - val_loss: 1.0312 - val_acc: 0.0500
Epoch 7/800
180/180 [==============================] - 0s 366us/step - loss: 1.0978 - acc: 0.2389 - val_loss: 0.9676 - val_acc: 0.0500
Epoch 8/800

'[test_loss, test_acc]'

[5.602794224875314, 0.276190476332392]